In [1]:
!pip install -q langdetect jsonlines

You should consider upgrading via the '/home/paul/anaconda3/bin/python -m pip install --upgrade pip' command.


In [2]:
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
import pandas as pd
import numpy as np
import torch
from octis.evaluation_metrics.diversity_metrics import TopicDiversity
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel
import umap
import optuna
import hdbscan
from tqdm.notebook import tqdm
import nltk
import matplotlib.pyplot as plt
from pylatexenc.latex2text import LatexNodes2Text
from razdel import tokenize
from nltk.corpus import stopwords
from string import punctuation
from natasha import Doc, MorphVocab, Segmenter, NewsEmbedding, NewsMorphTagger
from langdetect import detect


plt.style.use('seaborn')
tqdm.pandas()

In [3]:
def get_crps(txts: list):
    ret = []
    for el in txts:
        ret.append(el.split())
    return ret


def get_dict(data):
    return dict(zip(data['Topic'].tolist(), [el.split('_')[1:] for el in data['Name'].tolist()]))


def transform_topics(lst, dct):
    ret = []
    for el in lst:
        ret.append(dct[el])
    return ret


def compute_coherence(topic_model_, name_c, data, topics_, topn):
    cleaned_docs = topic_model_._preprocess_text(data)

    # Extract vectorizer and tokenizer from BERTopic
    vectorizer = topic_model_.vectorizer_model
    tokenizer = vectorizer.build_tokenizer()

    # Extract features for Topic Coherence evaluation
    words = vectorizer.get_feature_names()
    tokens = [tokenizer(doc) for doc in cleaned_docs]
    dictionary = corpora.Dictionary(tokens)
    corpus = [dictionary.doc2bow(token) for token in tokens]
    topic_words = [[words for words, _ in topic_model_.get_topic(topic) if words!='']
                   for topic in range(len(set(topics_))-1)]

    # Evaluate
    coherence_model = CoherenceModel(topics=topic_words,
                                     texts=tokens,
                                     corpus=corpus,
                                     dictionary=dictionary,
                                     coherence=name_c, topn=topn)
    coherence = coherence_model.get_coherence()
    return coherence


@torch.no_grad()
def compute_metrics(topics_, topic_model_, data, top_k=5):
    try:
        output = {'topics' : [[words for words, _ in topic_model_.get_topic(topic)]for topic in range(len(set
                                                                                                          (topics_[0]))-1)]}
        topic_diversity = TopicDiversity(topk=top_k)
        topic_diversity_score = topic_diversity.score(output)
    except:
        topic_diversity_score = None

    npmi_score = compute_coherence(topic_model_,'c_npmi', data, topics_[0], top_k)
    cv_score = compute_coherence(topic_model_,'c_v', data, topics_[0], top_k)

    return topic_diversity_score, npmi_score, cv_score


@torch.no_grad()
def objective(trial):
    torch.cuda.empty_cache()
    tnw = trial.suggest_int("top_n_words", 10, 30, log=True)
    ngr = trial.suggest_int("n_gram_range", 1, 3, log=True)
    mts = trial.suggest_int("min_topic_size", 5, 50, log=True)

    min_dist = trial.suggest_float("min_dist", 0.000001, 1, log=True)
    n_neigh = trial.suggest_int("n_neighbors", 2, 100, log=True)
    n_comp = trial.suggest_int("n_components", 10, 250, log=True)
    umap_model = umap.UMAP(
        n_neighbors=n_neigh,
        min_dist=min_dist,
        n_components=n_comp,
        random_state=42,
    )
    # cse = trial.suggest_float("cluster_selection_epsilon", 0.0001, 10, log=True)
    # mcs = trial.suggest_int("min_cluster_size", 2, 100, log=True)
    ms = trial.suggest_int("min_samples", 2, 40, log=True)
    hdbscan_model = hdbscan.HDBSCAN(min_samples=ms)
    topic_model = BERTopic(embedding_model=sentence_model, top_n_words=tnw, n_gram_range=(1, ngr), min_topic_size=mts,
                           umap_model=umap_model, hdbscan_model=hdbscan_model, verbose=False)


    topics = topic_model.fit_transform(X_train)
    return compute_metrics(topics, topic_model, X_train)[1]

def extract_pars(dct):
    tmp = dict()
    tmp['min_dist'] = dct['min_dist']
    tmp['n_neighbors'] = dct['n_neighbors']
    tmp['n_components'] = dct['n_components']
    tmp_1 = dict()
    # tmp_1['cluster_selection_epsilon'] = dct['cluster_selection_epsilon']
    # tmp_1['min_cluster_size'] = dct['min_cluster_size']
    tmp_1['min_samples'] = dct['min_samples']
    tmp_2 = dict()
    tmp_2['top_n_words'] = dct['top_n_words']
    tmp_2['n_gram_range'] = (1, dct['n_gram_range'])
    tmp_2['min_topic_size'] = dct['min_topic_size']
    return tmp, tmp_1, tmp_2

In [ ]:
/home/paul/Documents/ВКР/ru_kw_eval_datasets-master/data/cyberleninka_0.jsonlines.zip

In [9]:
from os import walk
filenames = next(walk('ru_kw_eval_datasets-master/data/'), (None, None, []))[2]

In [10]:
filenames

['habrahabr_0.jsonlines.zip',
 'russia_today_5.jsonlines.zip',
 'russia_today_0.jsonlines.zip',
 'cyberleninka_4.jsonlines.zip',
 'cyberleninka_1.jsonlines.zip',
 'cyberleninka_3.jsonlines.zip',
 'russia_today_2.jsonlines.zip',
 'ng_0.jsonlines.zip',
 'habrahabr_2.jsonlines.zip',
 'russia_today_7.jsonlines.zip',
 'ng_1.jsonlines.zip',
 'habrahabr_3.jsonlines.zip',
 'russia_today_6.jsonlines.zip',
 'russia_today_3.jsonlines.zip',
 'cyberleninka_2.jsonlines.zip',
 'cyberleninka_0.jsonlines.zip',
 'russia_today_1.jsonlines.zip',
 'habrahabr_1.jsonlines.zip',
 'russia_today_4.jsonlines.zip']

In [12]:
import zipfile
for fl in filenames:
    tmp = 'ru_kw_eval_datasets-master/data/' + fl
    with zipfile.ZipFile(tmp, 'r') as zip_ref:
        zip_ref.extractall('ru_data/')

# Habr

In [13]:
import jsonlines
habr = []
for fl in [f'ru_data/habrahabr_{i}.jsonlines' for i in range(4)]:
    with jsonlines.open(fl, 'r') as jsonl_f:
        habr += [obj for obj in jsonl_f]
len(habr)

3990

In [14]:
habr[0]

{'content': ' MassTransit это open source библиотека, разработанная на языке C# для .NET платформы, упрощающая работу с шиной данных, которая используется при построении распределенных приложений и реализации SOA (service oriented architecture). \r\nВ качестве message broker могут выступать RabbitMq, Azure Service Bus или In-Memory менеджер (в случае с In-Memory область видимости ограничивается процессом, в котором проинициализирован экземпляр). Содержание: Команды и события Команды \r\n События \r\n \r\n Контракты сообщений \r\n Роутинг Exchange \r\n Формат сообщения \r\n \r\n Консьюмеры (Consumer) \r\n Конфигурация контейнера DI \r\n Наблюдатели (Observer) \r\n Новое в MassTransit 3.0 \r\n Заключение \r\n Опрос: А какую .NET библиотеку используете вы? \r\n Команды и события \r\nВ библиотеке заложено 2 основных типа сообщений: команды и события. Команды \r\nСигнализируют о необходимости выполнить некое действие. Для наиболее содержательного наименования команды желательно использовать

In [27]:
habr[2]

{'content': ' \r\nНа сегодняшний день процедура реализации «failover» в Postgresql является одной из самых простых и интуитивно понятных. Для ее реализации необходимо определиться со сценариями файловера — это залог успешной работы кластера, протестировать его работу. В двух словах — настраивается репликация, чаще всего асинхронная, и в случае отказа текущего мастера, другая нода(standby) становится текущем «мастером», другие ноды standby начинают следовать за новым мастером. \r\nНа сегодняшний день repmgr поддерживает сценарий автоматического Failover — autofailover, что позволяет поддерживать кластер в рабочем состоянии после выхода из строя ноды-мастера без мгновенного вмешательства сотрудника, что немаловажно, так как не происходит большого падения UPTIME. Для уведомлений используем telegram. \r\n Появилась необходимость в связи с развитием внутренних сервисов реализовать систему хранения БД на Postgresql + репликация + балансировка + failover(отказоустойчивость). Как всегда в инте

In [15]:
urls_habr = [x['url'] for x in habr]
summaries_habr = [x['summary'] for x in habr]

In [16]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /home/paul/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/paul/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/paul/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [17]:
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

def natasha_lemmatize(text):
    text = text.lower()
    try:
        text = LatexNodes2Text().latex_to_text(text)
    except:
        text = text.lower()
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
    return [_.lemma for _ in doc.tokens]

def cleaner(txt, noise):
    res = []
    for word in txt:
        if word not in noise:
            res.append(word)
    return ' '.join(res)

def prep(txt_list):
    noise = stopwords.words('russian') + list(punctuation) + ['что', 'то', 'кто', 'привет', 'весь', 'всем', 'какой', 'ваш', 'внимание', 'добрый', 'время', 'сатья', 'данный', 'хотеть', 'рассказать', 'посвятить', 'результат', 'результаты', 'представить', 'привести']
    for i in tqdm(range(len(txt_list))):
        txt_list[i] = cleaner(natasha_lemmatize(txt_list[i]), noise)



cleaned_summaries_habr = summaries_habr.copy()
prep(cleaned_summaries_habr)
summaries_habr[0], cleaned_summaries_habr[0]

  0%|          | 0/3990 [00:00<?, ?it/s]

('MassTransit это open source библиотека, разработанная на языке C# для .NET платформы, упрощающая работу с шиной данных, которая используется при построении распределенных приложений и реализации...',
 'masstransit это open source библиотека разработать язык c net платформа упрощать работа шина данные который использоваться построение распределить приложение реализация ...')

In [18]:
import jsonlines
cyberlink = []
for fl in [f'ru_data/cyberleninka_{i}.jsonlines' for i in range(5)]:
    with jsonlines.open(fl, 'r') as jsonl_f:
        cyberlink += [obj for obj in jsonl_f]
len(cyberlink)

4072

In [19]:
def split_lang(txt):
    rus = ''
    eng = ''
    for sent in txt.split('.'):
        try:
            if detect(sent) == 'ru':
                rus += sent + '.'
            else:
                eng += sent + '.'
        except:
            continue
    return rus, eng

abstract_rus_link = []
abstract_eng_link = []
url_cyber_leninka = []

for sup in tqdm(cyberlink, desc='extracting abstracts'):
    abstr = sup['abstract']
    r, e = split_lang(abstr)
    abstract_rus_link.append(r)
    abstract_eng_link.append(e)
    url_cyber_leninka.append(sup['url'])

extracting abstracts:   0%|          | 0/4072 [00:00<?, ?it/s]

In [20]:
cleaned_abstract_r_link = abstract_rus_link.copy()
prep(cleaned_abstract_r_link)
abstract_rus_link[0], cleaned_abstract_r_link[0]

  0%|          | 0/4072 [00:00<?, ?it/s]

('Изложен метод проектирования устройств подачи нитки верхнего петлителя и определения рационального процесса ее потребления на основании использования диаграммы согласования функций подачи и потребления нитки верхнего петлителя при образовании трехниточного краеобметочного стежка 504 типа. Алгоритм, представленный для расчета диаграммы потребления нитки верхнего петлителя с учетом вероятных вариантов процесса взаимодействия рабочих органов, материала и нитки верхнего петлителя с ниткой нижнего петлителя, в совокупности с математическим аппаратом, используемый при этом, позволяет автоматизировать одну из особо сложных стадий в проектировании краеобметочной машины. Данный метод проектирования нитеподачи нитки верхнего петлителя будет полезен для разработчиков и эксплуатационников .',
 'изложить метод проектирование устройство подача нитка верхний петлитель определение рациональный процесс потребление основание использование диаграмма согласование функция подача потребление нитка верхний

In [30]:
abstract_rus_link[2]

''

In [21]:
data_comb = np.array(cleaned_abstract_r_link + cleaned_summaries_habr)
data_unck_comb = abstract_rus_link + summaries_habr
url_comb = url_cyber_leninka + urls_habr

In [26]:
data_comb[2]

''

In [25]:
len(url_comb), len(data_comb)

(8062, 8062)

In [22]:
df_rus = pd.DataFrame({'data_clean' : data_comb, 'data_unclean' : data_unck_comb, 'url' : url_comb})
df_rus.to_csv('habr_cyberleninka.csv')

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_comb, np.arange(data_comb.shape[0]), test_size=0.3,
                                                    random_state=42)

In [ ]:
model_name = 'paraphrase-multilingual-mpnet-base-v2'
sentence_model = SentenceTransformer(model_name, device="cuda")
study = optuna.create_study(sampler=optuna.samplers.CmaEsSampler(), direction="maximize")
study.enqueue_trial(
    {
        'top_n_words': 10,
        'n_gram_range': 1,
        'min_topic_size': 10,
        'min_dist': 0.000001,
        'n_neighbors': 15,
        'n_components': 5,
        # 'cluster_selection_epsilon': 0.0,
        # 'min_cluster_size': 5,
        'min_samples': 5
    }
)
study.optimize(objective, n_trials=50, n_jobs=1)
params_ = extract_pars(study.best_trial.params)

# sentence_model = SentenceTransformer(model_name, device="cuda")
umap_model = umap.UMAP(**params_[0], random_state=42)
hdbscan_model = hdbscan.HDBSCAN(**params_[1])
topic_model = BERTopic(embedding_model=sentence_model, umap_model=umap_model, hdbscan_model=hdbscan_model, **params_[2])

topics = topic_model.fit_transform(X_test)
compute_metrics(topics, topic_model, X_test)

In [ ]:
umap_model = umap.UMAP(**params_[0], random_state=42)
hdbscan_model = hdbscan.HDBSCAN(**params_[1], prediction_data = True)
topic_model = BERTopic(embedding_model=sentence_model, umap_model=umap_model, hdbscan_model=hdbscan_model,
                       **params_[2], verbose=True, calculate_probabilities=True)

topics = topic_model.fit_transform(data_comb)
compute_metrics(topics, topic_model, data_comb)

In [ ]:
# topic_model.save('best_rus')

In [ ]:
len(topic_model.get_topics())

In [ ]:
cv = []
npmi  = []
td = []
n_tops = list(range(100, 20, -10))
for nr in tqdm(range(100, 20, -10)):
    umap_model = umap.UMAP(**params_[0], random_state=42)
    hdbscan_model = hdbscan.HDBSCAN(**params_[1], prediction_data = True)
    topic_model = BERTopic(embedding_model=sentence_model, umap_model=umap_model, hdbscan_model=hdbscan_model,
                           **params_[2], verbose=True, calculate_probabilities=True)

    topics = topic_model.fit_transform(data_comb)
    new_topics, new_probs = topic_model.reduce_topics(data_comb, topics[0], topics[1],
                                                      nr_topics=nr)
    td_, npmi_, cv_ = compute_metrics((new_topics, new_probs), topic_model, data_comb)
    cv.append(cv_)
    td.append(td_)
    npmi.append(npmi_)

In [ ]:
import plotly.express as px


fig = px.line(x=n_tops, y=cv)
fig.show()

In [ ]:
fig = px.line(x=n_tops, y=npmi)
fig.show()

In [ ]:
fig = px.line(x=n_tops, y=td)
fig.show()

In [ ]:
best_nr = 70
umap_model = umap.UMAP(**params_[0], random_state=42)
hdbscan_model = hdbscan.HDBSCAN(**params_[1], prediction_data = True)
topic_model = BERTopic(embedding_model=sentence_model, umap_model=umap_model, hdbscan_model=hdbscan_model,
                       **params_[2], verbose=True, calculate_probabilities=True)

topics = topic_model.fit_transform(data_comb)
new_topics, new_probs = topic_model.reduce_topics(data_comb, topics[0], topics[1],
                                                  nr_topics=best_nr)

df = pd.DataFrame({'uncleaned texts' : data_unck_comb, 'texts' : data_comb, 'urls' : url_comb})
df['topics'] = new_topics
df.to_csv('best_so_far.csv')
topic_model.save('best_model_so_far')

In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_distribution(new_probs[200], min_probability=0.015)

In [ ]:
topic_model.visualize_hierarchy(top_n_topics=50)

In [ ]:
topic_model.visualize_barchart(top_n_topics=8)

In [ ]:
topic_model.visualize_term_rank()